In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Any results you write to the current directory are saved as output.

from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras import backend as K


##########################################################
# DATA PREPROCESSING
##########################################################

max_features = 20000 # 20000 
maxlen = 100 # 100


list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
#list_classes = ["toxic"]

train = pd.read_csv("data/train.csv")
tokenizer = text.Tokenizer(num_words=max_features)



#Creation of training data
train = train.sample(frac=1)
list_sentences_train = train["comment_text"].fillna("CVxTz").values
tokenizer.fit_on_texts(list(list_sentences_train))
y = train[list_classes].values
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)



#Creation of testing data
test = pd.read_csv("data/test.csv")
list_sentences_test = test["comment_text"].fillna("CVxTz").values
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)


##########################################################
# MODEL
##########################################################

def get_model():
    embed_size = 32
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Dropout(0.125)(x)
    x = Bidirectional(LSTM(16, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.125)(x)
    x = Dense(8, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(8, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model


model = get_model()

batch_size = 256
epochs = 10

model.summary()


file_path="weights_base_balanced.hdf5"

model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

'''model.load_weights(file_path)
'''
print("Predicting...")
y_test = model.predict(X_te)
print("done")



Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 32)           640000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 32)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 32)           6272      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 264       
__________

In [ ]:
submission = pd.read_csv('data/sample_submission.csv')
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_test
submission.to_csv('sub4.csv', index=False)


#Creation of oof data
train2 = pd.read_csv("data/train.csv")
list_sentences_train2 = train2["comment_text"].fillna("CVxTz").values
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_train2)
X_train2 = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

y_oof = model.predict(X_train2)
oof = pd.DataFrame
k = 0
for label in ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]:
    oof[label] = y_oof[:,k]
    k+=1
oof.to_csv("oof4.csv", index = False)

In [5]:
print("Predicting...")
y_test = model.predict(X_te)
print("done")

# ROC AUC analysis of results

from sklearn import metrics

# Données pour les Bidirectional LSTM original
i=0
for label in list_classes:
    print("\n ===== %s ======"%label)
    toto = [elem[i] for elem in y_known]
    toto2 = [elem[i] for elem in y_test]
    i+=1
    fpr, tpr, thresholds = metrics.roc_curve(toto, toto2, pos_label=1)
    print(metrics.auc(fpr, tpr))

Predicting...
done

 ===== toxic ======
0.969311351672

 ===== severe_toxic ======
0.985508628406

 ===== obscene ======
0.982135331987

 ===== threat ======
0.942346708467

 ===== insult ======
0.976315039388

 ===== identity_hate ======
0.954897026015


In [2]:
# Données pour le Bidirectional LSTM avec des données balancées sur 'toxic'
from sklearn import metrics
i=0
for label in list_classes:
    print("\n ===== %s ======"%label)
    toto = [elem[i] for elem in y_known]
    toto2 = [elem[i] for elem in y_test]
    i+=1
    fpr, tpr, thresholds = metrics.roc_curve(toto, toto2, pos_label=1)
    print(metrics.auc(fpr, tpr))


 ===== toxic ======
0.486944416074

 ===== severe_toxic ======
0.539682858346

 ===== obscene ======
0.527678877382

 ===== threat ======
0.423447242712

 ===== insult ======
0.505246324273

 ===== identity_hate ======
0.472312777716


In [11]:
from sklearn import metrics
i=0
for label in list_classes:
    print("\n ===== %s ======"%label)
    toto = [elem[i] for elem in y_known]
    
    toto2 = [1 for i in range(sum(toto))] +
    i+=1
    fpr, tpr, thresholds = metrics.roc_curve(toto, toto2, pos_label=1)
    print(metrics.auc(fpr, tpr))


 ===== toxic ======
0.5

 ===== severe_toxic ======
0.5

 ===== obscene ======
0.5

 ===== threat ======
0.5

 ===== insult ======
0.5

 ===== identity_hate ======
0.5


In [9]:
y_test[1]

array([ 0.78437555,  0.10807855,  0.32847658,  0.01657154,  0.32705757,
        0.0392561 ], dtype=float32)

In [58]:
j=0
for i in range(len(toto2)):
    if toto[i]==1 and toto3[i]<0.1:
        print("\n========= %i ==============="%i)
        print(test['comment_text'][140000+i])
        j+=1
    if j==10:
        break


========= 1177 ===============
THE FUKKING RETARDO RACISTO KANUCK

========= 2689 ===============
You are such a fucking faggot, there's no need for references for something that can't really be reliably cited. Even if there was a sociological study on people who have APD, coke drinkers, and cocaine users, it wouldn't be very reliable because there's no way to tell if people are lying. Fuck off you beaurocratic official peice of pedophile, and that citing and copyright shit has got to go. I refused to make bibliographies in high school for one reason: I never claimed that the work was mine, and I shouldn't have to claim who's it was because it's self-explanitory that I didn't make it since I didn't claim to have done so.

========= 2719 ===============
NOT SET IN NEW YORK !!!! 
why when i deleted the Category Video games set in New York City some retardeds keep putting it back ? now for all the retards who think it should be back listen up-the game takes place in liberty city not new 

In [59]:
y_known[8108]

array([1, 1, 0, 0, 0, 0])

In [55]:
list_classes

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [3]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [4]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

InvalidArgumentError: Cannot assign a device for operation 'MatMul': Operation was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device.
	 [[Node: MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/device:GPU:0"](a, b)]]

Caused by op 'MatMul', defined at:
  File "/home/maxime/anaconda3/envs/amld/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/maxime/anaconda3/envs/amld/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/maxime/anaconda3/envs/amld/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/maxime/anaconda3/envs/amld/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/maxime/anaconda3/envs/amld/lib/python3.5/site-packa

In [1]:
#  https://www.kaggle.com/CVxTz/keras-bidirectional-lstm-baseline-lb-0-051/code

import os
os.chdir("/home/maxime/Programmation/ToxicChallenge")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "."]).decode("utf8"))

# Any results you write to the current directory are saved as output.

from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras import backend as K


##########################################################
# DATA PREPROCESSING
##########################################################

max_features = 20000 # 20000 
maxlen = 100 # 100


train = pd.read_csv("data/train.csv")
train = train[:140000]
test = pd.read_csv("data/test.csv")
train = train.sample(frac=1)

list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").values


tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)


##########################################################
# MODEL
##########################################################

def get_model():
    embed_size = 32
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Dropout(0.125)(x)
    x = Bidirectional(LSTM(16, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.125)(x)
    x = Dense(8, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(8, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model


model = get_model()
batch_size = 2048
epochs = 5

model.summary()


file_path="weights_base_best3.hdf5"


##########################################################
# TRAINING
##########################################################

checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", patience=20)


callbacks_list = [checkpoint, early, TensorBoard(log_dir='/tmp/autoencoder')]
# tensorboard --logdir=/tmp/autoencoder
model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)



# serialize model to JSON
#model_json = model.to_json()
#with open("model.json", "w") as json_file:
#    json_file.write(model_json)
# serialize weights to HDF5
#model.save_weights("c_autoencoder.h5")
#print("Saved model to disk")

##########################################################
# PREDICTING
##########################################################

model.load_weights(file_path)

print("Predicting...")
y_test = model.predict(X_te)
print("done")

print("writing csv")
#sample_submission = pd.read_csv("sample_submission.csv")
sample_submission = pd.DataFrame()

sample_submission[list_classes] = y_test

print("...")
sample_submission.to_csv("baseline.csv", index=False)



Access_to_dataFrame.ipynb
bidirectional_lstm_balanced.py
bidirectional_lstm.py
bidir_lstm.ipynb
check_gpu.py
data
Easy_Classifiers.ipynb
Normal_messages.ipynb
Pooled GRU
preprocessing.ipynb
preprossessing.py
Simple_vect.ipynb
test_bid_lstm.ipynb
Toxic_1.ipynb
Untitled.ipynb
use_bidirectional_lstm.py
weights_base_balanced.hdf5
weights_base.best2.hdf5



Using TensorFlow backend.
/home/maxime/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 32)           640000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 32)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 32)           6272      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 264       
__________

AttributeError: module 'pandas' has no attribute 'dataFrame'

In [2]:

model.load_weights(file_path)

print("Predicting...")
y_test = model.predict(X_te)
print("done")

print("writing csv")
#sample_submission = pd.read_csv("sample_submission.csv")
sample_submission = pd.DataFrame()

sample_submission[list_classes] = y_test

print("...")
sample_submission.to_csv("baseline.csv", index=False)

Predicting...
done
writing csv


KeyError: "['toxic' 'severe_toxic' 'obscene' 'threat' 'insult' 'identity_hate'] not in index"

In [8]:
len(y_test[:,1])

153164